<a href="https://colab.research.google.com/github/Komsomolochka/stats-for-data-analysis/blob/main/bioinformatics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Дифференциально экспрессированные гены

In [88]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.stats.multitest as smm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Описание используемых данных

Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор  генов, которые позволили  бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей. 

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

In [3]:
data = pd.read_csv('/content/gene_high_throughput_sequencing.csv')

In [44]:
data.head()

,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,C1orf159,LOC254099,TTLL10,TNFRSF18,TNFRSF4,SDF4,B3GALT6,FAM132A,UBE2J2,SCNN1D,ACAP3,PUSL1,CPSF3L,GLTPD1,DVL1,MXRA8,AURKAIP1,CCNL2,LOC148413,MRPL20,LOC441869,VWA1,ATAD3C,ATAD3B,ATAD3A,C1orf70,SSU72,LOC643988,MIB2,...,RENBP,HCFC1,TMEM187,IRAK1,MECP2,TKTL1,FLNA,EMD,RPL10,SNORA70,DNASE1L1,TAZ,ATP6AP1,GDI1,FAM50A,PLXNA3,LAGE3,UBL4A,SLC10A3,FAM3A,G6PD,GAB3,DKC1,MPP1,F8,FUNDC2,MTCP1NB,MTCP1,VBP1,RAB39B,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,26.329928,57.553312,42.455733,6.614939,7.786382,14.283751,18.752091,65.825821,17.471334,14.135344,42.949564,35.946652,50.945017,19.738267,52.172044,34.753563,27.810055,83.303689,59.428091,42.306459,13.209938,20.161089,1.257614,66.621905,3.772842,40.210889,30.278655,14.576032,73.168939,34.692832,36.470811,...,3.164926,20.369209,5.577156,22.438224,20.981186,1.257614,146.983123,33.074695,4.761250,5.185276,15.282298,24.761483,66.017756,51.888333,40.000566,19.950798,19.306187,26.007591,10.139210,40.105866,43.047651,4.761250,30.624906,9.714340,5.577156,4.761250,10.139210,2.408148,12.214621,3.164926,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,39.140813,119.572567,47.367114,4.567931,8.049497,31.421187,19.744990,123.120249,20.615707,16.865244,73.826942,45.474497,63.043023,28.021783,62.483096,60.409871,41.747570,124.715481,128.770634,33.452597,17.876777,28.947097,1.815112,113.503762,1.815112,45.954959,36.946925,15.222429,127.777507,66.567847,42.166365,...,12.355210,22.255175,9.075559,38.347144,21.245338,1.815112,152.191825,49.097571,8.577883,1.815112,23.965891,30.426817,160.727095,82.216081,60.045184,40.681726,70.040738,44.200859,13.379397,60.045184,72.566645,1.815112,21.451109,6.199787,1.815112,6.199787,5.445335,3.475674,9.075559,1.815112,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,48.144685,55.579752,41.741942,3.978294,6.232790,24.317644,28.662801,64.494789,14.543176,12.407734,37.531155,32.074035,63.596176,13.303109,42.016789,26.142516,22.853562,76.118706,56.095106,40.053273,13.303109,20.461953,2.077597,58.702060,2.077597,42.153541,30.416278,16.750119,70.137476,35.481715,43.761239,...,5.228503,20.178729,3.978294,20.741310,22.085154,3.978294,145.308037,27.008755,6.232790,7.096343,16.750119,24.553171,58.702060,46.192467,44.154018,20.178729,13.728915,23.839609,7.865653,37.684188,39.474334,5.228503,38.139602,9.213541,6.232790,2.077597,12.407734,7.096343,10.387983,6.232790,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,27.076621,54.454714,35.933084,2.066576,7.823932,24.422935,15.963105,54.245171,10.870009,14.854454,23.227927,27.905160,49.869762,8.520713,31.903907,16.999606,20.631293,34.477366,39.983492,32.784180,10.870009,24.884876,2.066576,53.398778,9.164670,40.125656,21.968009,14.066895,39.983492,28.907458,27.700348,...,2.066576,13.656094,7.058702,16.661272,7.823932,2.066576,90.135274,17.976444,2.066576,9.164670,13.656094,15.602310,53.611629,24.654987,35.933084,3.957193,6.199729,11.871578,9.164670,32.434938,25.112660,2.066576,5.200770,14.466035,7.823932,7.823932,17.656837,2.066576,17.331337,5.200770,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,35.896701,48.134682,46.507417,7.179340,8.151497,23.096282,29.007823,78.309425,16.586217,18.698195,42.135928,39.393867,65.111910,15.819691,60.974762,27.872385,38.046667,86.335695,59.781684,56.754932,10.304894,19.857856,2.613616,59.197111,1.364917,59.740119,35.47

In [46]:
data.shape

(72, 15749)

### Часть 1:  применение t-критерия Стьюдента




В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:



*   для групп normal (control) и early neoplasia (treatment)
*   для групп early neoplasia (control) и cancer (treatment)





В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [49]:
data['Diagnosis'].value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [51]:
# делим данные на 3 выборки
normal = data[data['Diagnosis'] =='normal'].drop('Diagnosis',axis=1)
neoplasia = data[data['Diagnosis'] =='early neoplasia'].drop('Diagnosis',axis=1)
cancer = data[data['Diagnosis'] =='cancer'].drop('Diagnosis',axis=1)

In [58]:
# задаем функцию расчета t-критерия Стьюдента для каждого гена
def ttest_gens(control, treatment):
    p_values = {}
    for gen in control.columns:
        p_values[gen] = stats.ttest_ind(control[gen], treatment[gen], equal_var=False)[1]
    return p_values

In [60]:
# проводим множественный тест для групп normal (control) и early neoplasia (treatment)
p_values_normal_en = ttest_gens(normal,neoplasia)

In [68]:
# создаем датасет
p_values_normal_en = pd.DataFrame.from_dict(p_values_normal_en, columns=['p'], orient='index')

In [83]:
# считаем количество статистически значимых отличий
count1 = p_values_normal_en[p_values_normal_en['p']<0.05].shape[0]

In [74]:
# проводим множественный тест для групп early neoplasia (control) и cancer (treatment)
p_values_en_cancer = ttest_gens(neoplasia,cancer)

In [77]:
# создаем датасет
p_values_en_cancer = pd.DataFrame.from_dict(p_values_en_cancer, columns=['p'], orient='index')

In [79]:
# считаем количество статистически значимых отличий
count2 = p_values_en_cancer[p_values_en_cancer['p']<0.05].shape[0]

In [80]:
# функция для сохранения ответа

def save_answers(answer, file_name):
    with open(file_name, "w") as fout:
        fout.write(str(answer))

In [87]:
# сохраняем ответы в файлы
save_answers(count1, "answer11.txt")
save_answers(count2, "answer12.txt")

### Часть 2: поправка методом Холма

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

Обратите внимание, что


*   применять поправку на множественную проверку  нужно  ко всем  значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия. 
*   при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

 



In [153]:
# поправка Бонферрони для множественной проверки групп normal (control) и early neoplasia (treatment)
reject, p_corrected, a1, a2 = smm.multipletests(p_values_normal_en['p'], alpha = 0.025, method = 'holm') 

In [154]:
p_values_normal_en['p_corrected'] = p_corrected
p_values_normal_en['reject'] = reject

In [147]:
# считаем среднее для каждого гена в группах
neoplasia_mean = neoplasia.mean()
normal_mean = normal.mean()
cancer_mean = cancer.mean()

In [134]:
# задаем функцию кратности изменения значения
def fold_change(C, T):
    if T > C:
        r = T/C
    else:
        r = -C/T
    return r

In [155]:
# задаем функцию расчета fold change для выборок
def fold_change_gens(control, treatment, data):
  sign_diff=[]
  for i, p in enumerate(data['reject']):
      if p==True:
          sign_diff.append(fold_change(control[i], treatment[i]))
  return sign_diff

In [157]:
# практическая значимость изменений для normal (control) и early neoplasia (treatment)
fold_change_gens(normal_mean, neoplasia_mean, p_values_normal_en)

[1.509785482044398, 1.9748676656368698]

In [166]:
# записываем ответ
answer21 = len(fold_change_gens(normal_mean, neoplasia_mean, p_values_normal_en))

In [158]:
# поправка Бонферрони для множественной проверки групп early neoplasia (control) и cancer (treatment)
reject, p_corrected, a1, a2 = smm.multipletests(p_values_en_cancer['p'], alpha = 0.025, method = 'holm') 

In [159]:
p_values_en_cancer['p_corrected'] = p_corrected
p_values_en_cancer['reject'] = reject

In [167]:
# практическая значимость изменений для early neoplasia (control) и cancer (treatment)
fold_change2 = fold_change_gens(neoplasia_mean, cancer_mean, p_values_en_cancer)

In [168]:
# записываем ответ
answer22 = sum([1 if abs(x)>1.5 else 0 for x in fold_change2])

In [169]:
answer22

77

In [170]:
# сохраняем ответы в файлы
save_answers(answer21, "answer21.txt")
save_answers(answer22, "answer22.txt")

### Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

 Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H_0
​
 , когда они есть, и будут чаще отклонять H_0
​
 , когда отличий нет).

В качестве ответа к этому заданию требуется  ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5. 

In [182]:
# поправка Бенджамини-Хохберга для множественной проверки групп normal (control) и early neoplasia (treatment)
reject, p_corrected, a1, a2 = smm.multipletests(p_values_normal_en['p'], alpha = 0.025, method = 'fdr_bh') 

In [183]:
p_values_normal_en['p_corrected'] = p_corrected
p_values_normal_en['reject'] = reject

In [184]:
# практическая значимость изменений для normal (control) и early neoplasia (treatment)
fold_changes_1 = fold_change_gens(normal_mean, neoplasia_mean, p_values_normal_en)

In [185]:
# записываем ответ
answer31 = sum([1 if abs(x)>1.5 else 0 for x in fold_changes_1])

In [177]:
# поправка Бенджамини-Хохберга для множественной проверки групп early neoplasia (control) и cancer (treatment)
reject, p_corrected, a1, a2 = smm.multipletests(p_values_en_cancer['p'], alpha = 0.025, method = 'fdr_bh') 

In [178]:
p_values_en_cancer['p_corrected'] = p_corrected
p_values_en_cancer['reject'] = reject

In [179]:
# практическая значимость изменений для early neoplasia (control) и cancer (treatment)
fold_changes_2 = fold_change_gens(neoplasia_mean, cancer_mean, p_values_en_cancer)

In [180]:
# записываем ответ
answer32 = sum([1 if abs(x)>1.5 else 0 for x in fold_changes_2])

In [186]:
# сохраняем ответы в файлы
save_answers(answer31, "answer31.txt")
save_answers(answer32, "answer32.txt")